## User Story:
The IT department of _**Plain Corp**_ has implemented a custom-built _Computer Management Solution (CMS)_ which comes with a feature that reports information about computers in their environment such as metadata, installed programs and logins. The current problem faced by the IT team is that the reports created by CMS do not answer all the questions they have about the computers in their environment. Luckily, they figured out that the CMS stores its raw data in a SQLite (v3.45.0)  _database (DB)_ file and that they could write custom SQL queries to report the information not provided within the CMS reports.

## Goal:
Your job will be to update this Jupyter Notebook with the SQL queries needed to meet the IT team’s reporting needs. It is recommended to use a Database IDE to help with query writing and data exploration.

## Directions:
Download the attached zip file which contains all the files you need to complete this assignment. Below is a description of each of the three files within the attached zip file:
- **cms.sqlite**: The database you must query from to complete this assignment.
- **cms_data_dictionary.txt**: A data dictionary describing the tables within this database.
- **GA Mission - Data Analyst.ipynb**: The Jupyter Notebook file you must update to complete the assignment. Rules to abide by while completing this assignment are listed within this Jupyter Notebook.

## Our Expectations
This assignment <ins>**should take you no longer than 2 hours**</ins> to complete. Make sure your outputs are visible in a web browser when uploaded to a public repository and that they are presented in a way that is understandable by someone with some technical experience but no programming experience. Extra rules to abide by are described in the included Juptyter Notebook file.

We would prefer that you share the code with us by pushing it to a public repository in the hosting provider of your choosing. If you do not want to upload it to a public repository then you can zip up the repository folder on your machine and send it to us via email. If you go that route, please make sure that the .git folder is included.

We recommend  that you use a Database IDE (e.g., DBeaver, Datagrip, etc.) to assist with query writing and data exploration. If you have any questions, please contact us and we will do our best to clarify anything that is unclear.

## Rules:
- The code should be stored in a git repository.
- With the exception of ***Pandas***, you can use any Python Packages you like.
- Do not edit the data in the **cms.sqlite** DB file.
- Your query results must be output in a _Pandas DataFrame (PD)_.
- Questions 4 and 5 are written answer questions where no code or SQL is necessary.
- Question 6 is a written answer question that may require code or SQL depending on your answer. This question can be thought of as a chance to share any observations you've made or perform a light EDA on the data. There are no right or wrong answers for this question.

# Questions

In [165]:
import pandas as pd
import re
import sqlite3

# Establish the connection to the cms.sqlite database.
cms_database = sqlite3.connect("cms.sqlite")

# Ensure that all columns are displayed on one line.
pd.set_option("display.expand_frame_repr", False)

 <font size='5'> 1. Count how many Admin and Regular computers are in the database.</font>

In [172]:
# SQL query to retrieve the number of computers whose computer type is either 'Admin' or 'Regular'.

query = """
SELECT
    COUNT(*) AS 'Number of Admin and Regular Computers'
FROM computers WHERE computer_type = 'Admin' OR computer_type = 'Regular'
"""
num_admin_and_regular = pd.read_sql_query(query, cms_database)

num_admin_and_regular

,Number of Admin and Regular Computers
0,32


<font size='5'>2a. Identify all non-admin computers with the programs **Bitwarden**, **Quickpass** or **1password** installed, please note that the listed program names may not match exactly to the program names in the database. Be sure to include the computer names, operating system, program names and program version in the PD output.</font>

In [180]:
# Query to retrieve all computers' (excluding duplicates) names, operating systens, program names, and
# program versions that are not admin computers and have either
# Bitwarden, Quickpass, or 1password installed.

query = """SELECT
DISTINCT
    computers.computer_name AS 'Computer Name',
    computers.operating_system AS 'Computer Operating System',
    installed_programs.program_name AS 'Program Name',
    installed_programs.program_version AS 'Program Version'
FROM installed_programs
JOIN computers ON installed_programs.computer_name = computers.computer_name
WHERE computers.computer_type != 'Admin'
AND (installed_programs.program_name LIKE '1%' OR installed_programs.program_name LIKE 'bit%'
OR installed_programs.program_name LIKE 'quick%')
"""

non_admin_computers_with_programs = pd.read_sql_query(query, cms_database)
non_admin_computers_with_programs

,Computer Name,Computer Operating System,Program Name,Program Version
0,DESKTOP-O23A1M7,Microsoft Windows 11 Pro,Quickpass,15.0.21225.01
1,LAPTOP-55433J3D,Microsoft Windows 11 Business,Quickpass,15.0.21225.01
2,PLAINCORP-1012,Microsoft Windows 11 Pro,1password,2.7.4
3,PLAINCORP-1014,Microsoft Windows 11 Business,1password,2.7.4
4,PLAINCORP-1014,Microsoft Windows 11 Business,Quickpass,15.0.18198.01
5,PLAINCORP-DRAKE,Microsoft Windows 11 Pro,Quickpass,15.0.21225.01
6,PLAINCORP-WILL-P,Microsoft Windows 11 Business,Quickpass,15.0.21225.01
7,WINDEV2303EVAL,Microsoft Windows 11 Enterprise Evaluation,Quickpass,15.0.21225.01


<font size='5'>2b. Report the latest version of each program and how many computers have said programs installed.</font>

In [195]:
# The below three queries retrieve the latest version of the given program that is installed on computers.

latest_version_1pass = """
SELECT
    program_version
FROM installed_programs
WHERE program_name LIKE '1pass%'
ORDER BY program_name DESC
LIMIT 1
"""

latest_version_quickpass = """
SELECT
    program_version
FROM installed_programs
WHERE program_name LIKE 'quick%'
ORDER BY program_name DESC
LIMIT 1
"""

latest_version_bitwarden = """
SELECT
    program_version
FROM installed_programs
WHERE program_name LIKE 'bit%'
ORDER BY program_name DESC
LIMIT 1
"""

onepass_version = pd.read_sql_query(latest_version_1pass, cms_database)
quickpass_version = pd.read_sql_query(latest_version_quickpass, cms_database)
bitwarden_version = pd.read_sql_query(latest_version_bitwarden, cms_database)

if len(onepass_version) > 0:
    onepass_version = onepass_version["program_version"][0]
else:
    onepass_version = "Not Installed"

if len(quickpass_version) > 0:
    quickpass_version = quickpass_version["program_version"][0]
else:
    quickpass_version = "Not Installed"

if len(bitwarden_version) > 0:
    bitwarden_version = bitwarden_version["program_version"][0]
else:
    bitwarden_version = "Not Installed"

# The three queries below count the number of computers (excluding duplicates) that have the given program installed.

num_installed_1pass = """
SELECT
    DISTINCT COUNT(program_name) AS 'number'
FROM installed_programs
WHERE program_name LIKE '1pass%'
"""

num_installed_quickpass = """
SELECT
    DISTINCT COUNT(program_name) AS 'number'
FROM installed_programs
WHERE program_name LIKE 'quick%'
"""

num_installed_bitwarden = """
SELECT
    DISTINCT count(program_name) AS 'number'
FROM installed_programs
WHERE program_name LIKE 'bit%'
"""

num_onepass_installs = pd.read_sql_query(num_installed_1pass, cms_database)["number"][0]
num_quickpass_installs = pd.read_sql_query(num_installed_quickpass, cms_database)["number"][0]
num_bitwarden_installs = pd.read_sql_query(num_installed_bitwarden, cms_database)["number"][0]

data = {"Latest Installed Program Version": [quickpass_version, onepass_version, bitwarden_version],
       "Number of Computers with Program Installed": [num_quickpass_installs, num_onepass_installs, num_bitwarden_installs]}

results = pd.DataFrame(data, index = ["Quickpass", "1password", "Bitwarden"])
results

,Latest Installed Program Version,Number of Computers with Program Installed
Quickpass,15.0.21225.01,24
1password,2.7.4,2
Bitwarden,Not Installed,0


<font size='5'> 2c. Report each program and the ratio of which computers out of all computers have said programs installed. Express the ratio in the format of “**X** out of **Y**“ computers and as a percentage rounded to the 2nd. Hint: To calculate the percentage, you will have to cast one of the numbers to a decimal by multiplying them by **1.0**.</font>

In [192]:
# Query to determine the total number of computers (excluding duplicates).

total_computers_query = """SELECT
DISTINCT COUNT(computer_name)
FROM computers
"""

num_computers = pd.read_sql_query(total_computers_query, cms_database)
num_computers = num_computers["COUNT(computer_name)"][0] * 1.0

ratio_1pass = round(num_onepass_installs/num_computers, 2)
ratio_quickpass = round(num_quickpass_installs/num_computers, 2)
ratio_bitwarden = round(num_bitwarden_installs/num_computers, 2)

num_computers = int(num_computers)

data = {"Ratio of Computers with Program": [f"{num_quickpass_installs} out of {num_computers}",
                                            f"{num_onepass_installs} out of {num_computers}",
                                            f"{num_bitwarden_installs} out of {num_computers}"],
       "Percentage of Computers with Program": [ratio_quickpass, ratio_1pass, ratio_bitwarden]}

results = pd.DataFrame(data, index = ["Quickpass", "1password", "Bitwarden"])
results

,Ratio of Computers with Program,Percentage of Computers with Program
Quickpass,24 out of 38,0.63
1password,2 out of 38,0.05
Bitwarden,0 out of 38,0.00


<font size='5'>3a. Report which admin computers have logins on or later than May 2023, have **Python** installed on their computers and whose IP addresses are not in the IP whitelist table. Be sure to include the computer name, IP address and login timestamp; order the results by most recent date and computer name.</font>

In [198]:
# Query to select computers (excluding duplicates) that are classified as Admin, have Python installed,
# the computer was used to login from May 2023 or later, and the IP address used to login was not
# found in the IP whitelist table. The results are sorted by date and time descending, and the computer name descending.

query = """SELECT
DISTINCT login_records.computer_name as 'Computer Name',
            login_records.login_ip as 'Login IP Address',
            login_records.login_timestamp as 'Date and Time of Login'
FROM login_records JOIN computers ON login_records.computer_name = computers.computer_name
JOIN installed_programs ON login_records.computer_name = installed_programs.computer_name
JOIN ip_whitelist ON ip_whitelist.login_ip != login_records.login_ip
WHERE computers.computer_type = "Admin" AND installed_programs.program_name LIKE "Python%"
AND login_timestamp >= "2023-05-01" AND login_records.login_ip NOT IN (ip_whitelist.login_ip)
ORDER BY login_records.computer_name DESC, login_records.login_timestamp DESC
"""

admin_computer_data = pd.read_sql_query(query, cms_database)
admin_computer_data

,Computer Name,Login IP Address,Date and Time of Login
0,PLAINCORP-13,73.18.119.169,2023-07-19 15:56:48
1,PLAINCORP-13,73.18.119.169,2023-07-19 15:56:45
2,DESKTOP-AFKOOLQ,75.19.119.2,2023-06-20 15:00:10
3,DESKTOP-AFKOOLQ,185.159.158.100,2023-05-16 23:22:29


<font size='5'>3b. Below are the requirements for the modifications you should make to the PD output from **3a**, you cannot change the SQL query to meet these requirements:</font>
- <font size='5'>Replace the 2nd and 4th octets of the IP address with **\*** (e.g., 10.\*.0.\*). Your solution must use regex.</font>
- <font size='5'>Format the login timestamp as the date only (e.g., 01/20/2024)</font>


In [204]:
def reformat_ip_address(ip_address):
    """
    Replaces the 2nd and 4th octets of the IP address with an asterisk.
    
    Example: 1.1.1.1 --> 1.*.1.*
    """
    
    ip_address = ip_address.split(".")
    pattern1 = fr'\b{ip_address[1]}\b'
    pattern2 = fr'\b{ip_address[3]}\b'

    new_second_octet = re.sub(pattern1, "*", ip_address[1])
    new_fourth_octet = re.sub(pattern2, "*", ip_address[3])
    return ".".join([ip_address[0], new_second_octet, ip_address[2], new_fourth_octet])

def format_date(date):
    """
    Returns the date in yyyy-mm-dd format.

    Example: 2023-07-19 15:56:48 --> 2023-07-19
    """
    
    return date[0:10]

revised_admin_data = admin_computer_data

# Rename column 'Date and Time of Login' to 'Login Date'.
revised_admin_data = revised_admin_data.rename(columns = {"Date and Time of Login": "Login Date"})

# Change the IP addresses to replace the 2nd and 4th octets with an asterisk.
revised_admin_data["Login IP Address"] = revised_admin_data["Login IP Address"].apply(reformat_ip_address)

# Change the dates to yyyy-mm-dd format.
revised_admin_data["Login Date"] = revised_admin_data["Login Date"].apply(format_date)

revised_admin_data

,Computer Name,Login IP Address,Login Date
0,PLAINCORP-13,73.*.119.*,2023-07-19
1,PLAINCORP-13,73.*.119.*,2023-07-19
2,DESKTOP-AFKOOLQ,75.*.119.*,2023-06-20
3,DESKTOP-AFKOOLQ,185.*.158.*,2023-05-16


<font size='5'>4. Bob from the IT department noticed a SQL query was starting to get slow and since he knew a little bit of SQL from college, he decided to fix the query himself. The first thing Bob did was to reduce the number of unnecessary joins in the query, but the new query was still slow. Was Bob’s initial approach the correct one? If so, explain Bob's next steps? If not, explain what Bob should've done instead?</font>

Bob's initial approach was correct.

His next steps would be to only include columns from the tables that are necessary, since it would take additional
time to retrieve data from the extraneous columns and splitting up the query into smaller queries. This is because
it would take less time to process each individual query, which could decrease the amount of time it takes to process all queries.

<font size='5'>5. To improve the CMS’s backend and report capabilities, the IT department wants to provide feedback to the contractor who built the CMS solution and wants your input.  How would you improve or change the data model? If you decide to create a data diagram you must save it as a PDF file in the same folder as this Jupyter Notebook.</font>

To change the data model, I would add a new column in the login_records table called "is_ip_address_on_ip_whitelist"
that has the value of true if a given IP address can be found in the login_ip column in the ip_whitelist table and false otherwise.

<font size='5'>6. Did you notice anything interesting or anything wrong with the data? If so, please describe what you found and if possible, provide any related evidence.</font>

In [205]:
# There was at least one instance where there were dulicate rows in the data.
# Here is an example, with the duplicates seen in rows 4 and 5 (excluding the login_record_id):

query = """SELECT * FROM login_records ORDER BY computer_name DESC LIMIT 10"""

results = pd.read_sql_query(query, cms_database)
results

,login_record_id,computer_name,login_ip,login_timestamp
0,69,WORK-CC,196.111.92.7,2023-10-26 19:28:03
1,50,WINDEV2303EVAL,99.183.189.1,2023-04-10 19:08:07
2,51,WINDEV2303EVAL,99.183.189.1,2023-04-10 19:08:11
3,38,WIN11-DUTCH,72.21.201.195,2023-01-12 21:23:42
4,39,WIN11-DUTCH,72.21.201.195,2023-01-12 21:23:43
5,40,WIN11-DUTCH,72.21.201.195,2023-01-12 21:23:43
6,41,WIN11-DUTCH,72.21.201.195,2023-01-12 21:23:45
7,42,WIN-4TLVKKSI3FQ,72.21.201.195,2022-05-19 18:20:27
8,20,TP-6806,74.105.105.185,2022-04-20 21:34:04
9,22,TP-6806,74.105.105.185,2022-04-13 12:49:08


In [ ]:
# Close the connection to the database.

cms_database.close()